## Нахождение неверно размеченных данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import re

import nltk
from nltk.stem.snowball import SnowballStemmer

import matplotlib.pyplot as plt
import matplotlib as mpl


from transliterate import translit

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

from scipy.stats import itemfreq

from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC

from sklearn.naive_bayes import MultinomialNB 

from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.neural_network import MLPClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import KFold

import pickle

In [17]:
df = pd.read_csv('evo_train.csv')

In [3]:
df_names = pd.read_csv('categories.csv')

In [4]:
df_names

,GROUP_ID,NAME,PATH
0,6,Продукты питания,Прод->Продукты питания
1,7,Табачные изделия\r,Непрод->Табачные изделия\r
2,9,н/д,н/д->н/д
3,12,"Бытовая техника, оборудование и электроника","Непрод->Бытовая техника, оборудование и электр..."
4,14,"Косметика, гигиена и парфюмерия","Непрод->Косметика, гигиена и парфюмерия"
5,15,Муз.инструменты,Непрод->Муз.инструменты
6,16,Канцтовары,Непрод->Канцтовары
7,17,Текстильные товары,Непрод->Текстильные товары
8,18,Спорттовары,Непрод->Спорттовары
9,19,Упаковка,Непрод->Упаковка


In [5]:
df.head()

,NAME,GROUP_ID,id
0,"Пиво ""Жигулевское"" 0,5 л. св.",35,0
1,СОУС ТОМАТНЫЙ БУЗДЯК 670Г ТАТАРСКИЙ /8,6,1
2,Сигареты Esse SS Exchange,7,2
3,Петрушка,6,3
4,пиво ягерь,35,4


In [7]:
df['GROUP_ID'].value_counts()

7     6045
6     6014
24    5975
14    5972
34    5963
35    5957
30    3050
26    2999
25    2961
29     639
28     633
9      626
37     618
17     616
18     613
16     612
20     608
59     607
27     606
19     604
22     603
21     587
52     586
60     584
36     572
12     568
38     394
15     235
32      13
Name: GROUP_ID, dtype: int64

In [9]:
# Делаем список стоп-слов.
stopwords = nltk.corpus.stopwords.words('russian')

In [10]:
# Стеммер.
stemmer = SnowballStemmer('russian')

In [11]:
def tokenizing_stemming(text):
    # Токенизация.
    tokens = [translit(word.lower(), 'ru') for word in nltk.word_tokenize(text, language='russian')]
    filtered_tokens = []
    # Возьмем слова содержащие только буквы.
    for token in tokens:
        if re.search('[а-яА-Яa-zA-Z0-9]', token):
            filtered_tokens.append(token.lower())
    # Стемминг.
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [12]:
# Кодируем в TF-IDF.
transformer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2),
                              use_idf=True,
                              tokenizer=tokenizing_stemming)
tfidf_items = transformer.fit_transform(df['NAME'])

За основу я взял эту статью: [Identifying Mislabeled Training Data](https://arxiv.org/pdf/1106.0219.pdf)

Сделаем кросс-валидацию на 10 фолдов и обучим бустинг над деревьями, случайный лес, логистическую регрессию и наивный байесовский классификатор.
Обучение длилось какое-то время, так что для надежности я сохранил натренированные модели и разбивку на обучающий и тест сеты для каждого разбиения.

In [251]:
kf = KFold(n_splits=10, shuffle=True)

In [252]:
model_boost = XGBClassifier()
model_rnd = RandomForestClassifier(n_estimators=30)
model_logreg = LogisticRegression()
model_multinb = MultinomialNB()

mod_numb = 0
for train_index, test_index in kf.split(tfidf_items):
    mod_numb += 1 
    mislabled = []
    
    train_index.dump('train_index{}.pkl'.format(mod_numb))
    test_index.dump('test_index{}.pkl'.format(mod_numb))
    
    model_boost.fit(tfidf_items[train_index], df['GROUP_ID'][train_index])
    model_rnd.fit(tfidf_items[train_index], df['GROUP_ID'][train_index])
    model_logreg.fit(tfidf_items[train_index], df['GROUP_ID'][train_index])
    model_multinb.fit(tfidf_items[train_index], df['GROUP_ID'][train_index])
    model_svc.fit(tfidf_items[train_index], df['GROUP_ID'][train_index])
    
    pkl_filename = 'model_boost{}.pkl'.format(mod_numb)  
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model_boost, file)
            
    pkl_filename = 'model_rnd{}.pkl'.format(mod_numb)  
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model_rnd, file)
            
    pkl_filename = 'model_logreg{}.pkl'.format(mod_numb)  
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model_logreg, file)
        
    pkl_filename = 'model_multinb{}.pkl'.format(mod_numb)  
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model_multinb, file)
        
    pkl_filename = 'model_svc{}.pkl'.format(mod_numb)  
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model_svc, file)

Найдем теперь как наши натренированные модели классифицировали данные в отложенных для теста фолдах. Кроме того, для каждого товара найдем вероятности, которые натренированные классификаторы присвоили тем классам (группам товаров), которыми они размечены. Мы будем использовать эти вероятности для выявления ошибочно размеченных товаров.

In [18]:
def data_set_label_proba(df, tfidf_items):
    
    # Добавление колонок в датафрейм, куда прочитаны данные.
    
    # Колонки для вероятностей, которые предсказали классификаторы группу товара.
    df['BOOST_PROB'] = None
    df['RND_PROB'] = None
    df['LOGREG_PROB'] = None
    df['MULTINB_PROB'] = None
    
    # Вспомогательные колонки. Используются, чтобы найти индекс нужной группы товаров в массиве предсказанных вероятностей
    # для каждого товара.
    df['BOOST_PROB_INDEX'] = None
    df['RND_PROB_INDEX'] = None
    df['LOGREG_PROB_INDEX'] = None
    df['MULTINB_PROB_INDEX'] = None
    
    # Предсказания сделанные каждым из классификаторов для каждого товара.
    df['BOOST_PRED'] = None
    df['RND_PRED'] = None
    df['LOGREG_PRED'] = None
    df['MULTINB_PRED'] = None
    
    for fold_number in range(1, 11):
        
        # Загрузка моделей из дампа для каждого фолда.
        with open('model_boost{}.pkl'.format(fold_number), 'rb') as file:  
            boost_pickle_model = pickle.load(file)

        with open('model_rnd{}.pkl'.format(fold_number), 'rb') as file:  
            rnd_pickle_model = pickle.load(file)

        with open('model_logreg{}.pkl'.format(fold_number), 'rb') as file:  
            logreg_pickle_model = pickle.load(file)
            
        with open('model_multinb{}.pkl'.format(fold_number), 'rb') as file:  
            multinb_pickle_model = pickle.load(file)
            
        # Индексы тестовых данных.
        test_index = np.load('test_index{}.pkl'.format(fold_number))
        
        # Вероятности каждой группы для каждого товара.
        boost_proba = boost_pickle_model.predict_proba(tfidf_items[test_index])
        rnd_proba = rnd_pickle_model.predict_proba(tfidf_items[test_index])
        logreg_proba = logreg_pickle_model.predict_proba(tfidf_items[test_index])
        multinb_proba = multinb_pickle_model.predict_proba(tfidf_items[test_index])
        
        # Предсказания классификаторов.
        boost_pred = boost_pickle_model.predict(tfidf_items[test_index])
        rnd_pred = rnd_pickle_model.predict(tfidf_items[test_index])
        logreg_pred = logreg_pickle_model.predict(tfidf_items[test_index])
        multinb_pred = multinb_pickle_model.predict(tfidf_items[test_index])
        
        # Нахождение и запись в датафрейм предсказанных вероятностей для группы, которой каждый товар размечен.
        for ind, val in enumerate(boost_pickle_model.classes_):
            index0 = df.iloc[test_index][(df['GROUP_ID'] == val)].index
            df.loc[index0, 'BOOST_PROB_INDEX'] = ind
            
        df.loc[test_index, 'BOOST_PROB'] = boost_proba[np.arange(np.shape(boost_proba)[0]), 
                                                        df.iloc[test_index]['BOOST_PROB_INDEX'].values.astype(int)]
        
                
        for ind, val in enumerate(rnd_pickle_model.classes_):
            index0 = df.iloc[test_index][(df['GROUP_ID'] == val)].index
            df.loc[index0, 'RND_PROB_INDEX'] = ind
            
        df.loc[test_index, 'RND_PROB'] = rnd_proba[np.arange(len(rnd_proba)), 
                                                        df.iloc[test_index]['RND_PROB_INDEX'].values.astype(int)]

        
        for ind, val in enumerate(logreg_pickle_model.classes_):
            index0 = df.iloc[test_index][(df['GROUP_ID'] == val)].index
            df.loc[index0, 'LOGREG_PROB_INDEX'] = ind
            
        df.loc[test_index, 'LOGREG_PROB'] = logreg_proba[np.arange(len(logreg_proba)), 
                                                        df.iloc[test_index]['LOGREG_PROB_INDEX'].values.astype(int)]

        
        for ind, val in enumerate(multinb_pickle_model.classes_):
            index0 = df.iloc[test_index][(df['GROUP_ID'] == val)].index
            df.loc[index0, 'MULTINB_PROB_INDEX'] = ind
            
        df.loc[test_index, 'MULTINB_PROB'] = multinb_proba[np.arange(len(multinb_proba)), 
                                                        df.iloc[test_index]['MULTINB_PROB_INDEX'].values.astype(int)]
        
        # Запись предсказаний каждой из моделей
        df.loc[test_index, 'BOOST_PRED'] = boost_pred
        df.loc[test_index, 'RND_PRED'] = rnd_pred
        df.loc[test_index, 'LOGREG_PRED'] = logreg_pred
        df.loc[test_index, 'MULTINB_PRED'] = multinb_pred

    return df

Будем считать неверно размеченным тот товар, у которого вероятности, предсказанные для его группы, меньше заданного порога для каждого классификатора. 

In [22]:
def mislabeled_check(df, boost_pr=0.1, rnd_pr=0.1, logreg_pr=0.1, multinb_pr=0.1):
    df['MISLABELED'] = False
    df.loc[(df['BOOST_PROB'] < boost_pr) & (df['RND_PROB'] < rnd_pr) & (df['LOGREG_PROB'] < logreg_pr) &
           (df['MULTINB_PROB'] < multinb_pr), 'MISLABELED'] = True
    
    return df

In [20]:
df_pr = data_set_label_proba(df, tfidf_items)

/home/vanya/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/vanya/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/vanya/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/vanya/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/vanya/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:83: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/vanya/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: Th

In [23]:
df_mis = mislabeled_check(df_pr)

Алгоритм предсказал 1742 неверно размеченных товара.

In [24]:
sum(df_mis['MISLABELED'])

1742

In [30]:
df_mis[df_mis['MISLABELED'] == True][['NAME', 'GROUP_ID', 'BOOST_PROB', 'RND_PROB', 'LOGREG_PROB', 'MULTINB_PROB',
                                    'BOOST_PRED', 'RND_PRED', 'LOGREG_PRED', 'MULTINB_PRED']]

,NAME,GROUP_ID,BOOST_PROB,RND_PROB,LOGREG_PROB,MULTINB_PROB,BOOST_PRED,RND_PRED,LOGREG_PRED,MULTINB_PRED
12,Масло 180г Азова,34,0.0366528,0,0.0175191,0.053927,6,6,6,6
59,тепловая завеса КЭВ 3П115Е 3кВт,22,0.0129517,0,0.0145481,0.0128187,6,9,6,6
81,Напульсники красные,59,0.0166986,0.0333333,0.0267887,0.0100129,6,7,34,34
86,BELCAT BC-950 тюнер хроматический,12,0.0149707,0,0.0177141,0.0117211,6,34,6,6
125,ликер Витторио Морелло,35,0.0322986,0,0.0315992,0.0647497,34,34,34,34
132,Телефон Maxvi P11 серебристый,36,0.00719107,0,0.0147618,0.0151288,6,34,6,7
141,8875 вызов специалиста,26,0.0668406,0,0.0661877,0.0555759,6,34,6,6
212,Вольтарен эм. гель 20.0,27,0.00881434,0,0.0231844,0.0168237,14,14,14,14
246,ПЛАТКИ НОСОВЫЕ БУМ. HELEN HARPER 10 ШТ,17,0.00666621,0.0666667,0.0210313,0.0108738,6,24,24,24
269,"МОРИЛКА ДУБ 0,5 Л НЕВОДНАЯ",28,0.0126113,0,0.0133137,0.00357128,6,35,35,35


### Замечения
1. Чтобы улучшить результат, возможно нужно сделать grid search по гиперпараметрам моделей.
2. Надо поиграть с пороговыми вероятностями в функции mislabeled_check(). 
3. Для проверки качества определения неверно размеченных товаров можно выбросить те, которые выбрал алгоритм, обучить классификаторы на оставшихся данных и посмотреть, улучшилась их предсказательная способность. 